In [ ]:
import numpy as np
from time import time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing 
from sklearn.datasets import fetch_lfw_people
import torch
import torch.nn as nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
import skorch
import torchvision

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [ ]:
X = lfw_people.data
n_features = X.shape[1]

the label to predict is the id of the person

In [ ]:
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_samples, h, w = lfw_people.images.shape

In [ ]:
print("Total dataset size:")
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=1, stratify=y)

In [ ]:
class Resnet18(nn.Module):
    """ResNet 18, pretrained, with one input chanel and 7 outputs."""
    def __init__(self, in_channels=1, n_outputs=7):
        super(Resnet18, self).__init__()

        # self.model = torchvision.models.resnet18()
        self.model = torchvision.models.resnet18(pretrained=True)

        # original definition of the first layer on the renset class
        # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
        #                        bias=False)
        # one channel input (greyscale):
        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2,
                                     padding=3, bias=False)

        # Last layer
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, n_outputs)
    def forward(self, x):
        return self.model(x)

In [ ]:
torch.manual_seed(0)
resnet = NeuralNetClassifier(
    Resnet18,
    # `CrossEntropyLoss` combines `LogSoftmax and `NLLLoss`
    criterion=nn.CrossEntropyLoss,
    max_epochs=50,
    batch_size=128,  # default value
    optimizer=torch.optim.Adam,
    # optimizer=torch.optim.SGD,
    optimizer__lr=0.001,
    optimizer__betas=(0.9, 0.999),
    optimizer__eps=1e-4,
    optimizer__weight_decay=0.0001,  # L2 regularization
    # Shuffle training data on each epoch
    # iterator_train__shuffle=True,
    train_split=skorch.dataset.CVSplit(cv=5, stratified=True),
    device=device,
    verbose=0)

In [ ]:
scaler = preprocessing.MinMaxScaler()
X_train_s = scaler.fit_transform(X_train).reshape(-1, 1, h, w)
X_test_s = scaler.transform(X_test).reshape(-1, 1, h, w)

In [ ]:
t0 = time()
resnet.fit(X_train_s, y_train)
print("done in %0.3fs" % (time() - t0))

Continue training a model (warm re-start):<br>
resnet.partial_fit(X_train_s, y_train)

In [ ]:
y_pred = resnet.predict(X_test_s)
print(classification_report(y_test, y_pred, target_names=target_names))